In [1]:
import pandas as pd

In [6]:
corpus_file = '../../data/democracy_all_predictions_020824.csv'
keyword_file = '../../data/democracy_keywordextraction_aggregated_counts_240724.csv'
df = pd.read_csv(corpus_file)
df_keyword = pd.read_csv(keyword_file)
#print(df)

/var/folders/pj/wj7m849s68qcg6_lr0j03jgh0000gn/T/ipykernel_44153/2593485023.py:3: DtypeWarning: Columns (4,5,6,8,9,10,12,13,14,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(corpus_file)


In [5]:
# Some checks on dataset
# Georgia 2006 GRECO has only 7 sentences
#scraped_sources = '../../data/democracy_reports_corpus_290724.csv'
#df_s = pd.read_csv(scraped_sources)
#filtered_df = df_s[(df_s['country'] == 'georgia') & (df_s['source'] == 'greco') & (df_s['year'] == 2006)]
#print(filtered_df)

In [99]:
# Agregate dimensions as contry, year source dimension
df_agg_roberta = df.groupby(['country', 'year', 'source', 'predicted_dim_roberta_unbalanced']).size().unstack(fill_value=0).stack().reset_index(name='dim_roberta_unbalanced_agg_count')
df_agg_roberta_no_ambiguous = df.groupby(['country', 'year', 'source', 'predicted_dim_roberta_no_ambiguous_unbalanced']).size().unstack(fill_value=0).stack().reset_index(name='dim_roberta_no_ambiguous_unbalanced_agg_count')
#df_agg_roberta_no_ambiguous_balanced = df.groupby(['country', 'year', 'source', 'predicted_dim_roberta_no_ambiguous_balanced']).size().reset_index(name='dim_roberta_no_ambiguous_balanced_agg_count')

# Sentiment groupings
#df_agg_roberta_sentiment = df.groupby(['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_unbalanced']).size().reset_index(name='backsliding_roberta_agg_count')
df_agg_roberta_sentiment = df.groupby(['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_unbalanced']).size().unstack(fill_value=0).stack().reset_index(name='backsliding_roberta_agg_count')
df_agg_roberta__no_ambiguous_sentiment = df.groupby(['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_no_ambiguous_unbalanced']).size().unstack(fill_value=0).stack().reset_index(name='backsliding_roberta_no_ambiguous_unbalanced_agg_count')
# Corpus size by country, source, year
df_agg_corpus = df.groupby(['country', 'year', 'source']).size().reset_index(name='sentence_count')


In [100]:
df_keyword = pd.read_csv(keyword_file)
df_keyword.rename(columns={'dimension': 'dim_keyword_dictionary', 'count': 'dim_keyword_dictionary_agg_count'}, inplace=True)
#print(df_keyword)

In [101]:
#df_temp_1 = pd.merge(df_agg_roberta, df_agg_roberta_no_ambiguous, on=['country', 'year', 'source'], how='left')
df_temp_1 = pd.merge(df_agg_roberta, df_agg_roberta_no_ambiguous, left_on=['country', 'year', 'source', 'predicted_dim_roberta_unbalanced'], right_on=['country', 'year', 'source','predicted_dim_roberta_no_ambiguous_unbalanced'], how='outer')
df_temp_2 = pd.merge(df_temp_1, df_keyword, left_on=['country', 'year', 'source', 'predicted_dim_roberta_unbalanced'], right_on=['country', 'year', 'source','dim_keyword_dictionary'], how='outer')

#df_temp_3 = pd.merge(df_temp_2, df_agg_roberta_sentiment, on=['country', 'year', 'source', 'predicted_dim_roberta'], how='left')
#df_temp_3 = pd.merge(df_temp_2, df_keyword, on=['country', 'year', 'source'], how='outer')
df_agg_all =  pd.merge(df_temp_2, df_agg_corpus, on=['country', 'year', 'source'], how='left')

In [102]:
def fill_blanks(row):
    if(pd.isna(row['predicted_dim_roberta_no_ambiguous_unbalanced'])):
        row['predicted_dim_roberta_no_ambiguous_unbalanced'] = row['predicted_dim_roberta_unbalanced']
        row['dim_roberta_no_ambiguous_unbalanced_agg_count'] = 0
    if(pd.isna(row['dim_keyword_dictionary'])):
        row['dim_keyword_dictionary'] = row['predicted_dim_roberta_unbalanced']
        row['dim_keyword_dictionary_agg_count'] = 0
    return row
def fill_blanks_back(row):
    if(pd.isna(row['predicted_dim_roberta_no_ambiguous_unbalanced'])):
        row['predicted_dim_roberta_no_ambiguous_unbalanced'] = row['predicted_dim_roberta_unbalanced']
        row['backsliding_roberta_no_ambiguous_unbalanced_agg_count'] = 0
    return row   

In [103]:
df_agg_all = df_agg_all.apply(fill_blanks, axis=1)

In [104]:
print(df_agg_roberta_sentiment.shape[0])

23628


In [105]:
df_back_1 = pd.merge(df_agg_roberta_sentiment, df_agg_roberta__no_ambiguous_sentiment, left_on=['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_unbalanced'], right_on=['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_no_ambiguous_unbalanced'], how='outer')
df_back_1 = df_back_1.apply(fill_blanks_back, axis=1)
df_back_1 = df_back_1.rename(columns={'predicted_backsliding_123': 'backsliding'})
df_agg_back =  pd.merge(df_back_1, df_agg_corpus, on=['country', 'year', 'source'], how='left')

In [106]:
#print(df_agg_back)
#print(df_back_1)

In [107]:
from datetime import datetime
file_name = datetime.now().strftime('../../data/democracy_aggregate_counts_for_backsliding_%d%m%y')
#file_name = datetime.now().strftime('temp_democracy_aggregate_counts_for_backsliding_%d%m%y')

print(f"Saving {file_name}.csv")
df_agg_back.to_csv(f"{file_name}.csv", index=False)
print(f"Saving {file_name}.xlsx")
df_agg_back.to_excel(f"{file_name}.xlsx", index=False)

Saving ../../data/democracy_aggregate_counts_for_backsliding_020824.csv
Saving ../../data/democracy_aggregate_counts_for_backsliding_020824.xlsx


In [108]:
test_df = pd.read_csv(f"{file_name}.csv")
print(test_df.columns)
any_nulls = test_df.isnull().values.any()

print("Any null values:", any_nulls)
# Get the count of null values in each column
null_counts = test_df.isnull().sum()

print("Count of null values in each column:\n", null_counts)


Index(['country', 'year', 'source', 'backsliding',
       'predicted_dim_roberta_unbalanced', 'backsliding_roberta_agg_count',
       'predicted_dim_roberta_no_ambiguous_unbalanced',
       'backsliding_roberta_no_ambiguous_unbalanced_agg_count',
       'sentence_count'],
      dtype='object')
Any null values: False
Count of null values in each column:
 country                                                  0
year                                                     0
source                                                   0
backsliding                                              0
predicted_dim_roberta_unbalanced                         0
backsliding_roberta_agg_count                            0
predicted_dim_roberta_no_ambiguous_unbalanced            0
backsliding_roberta_no_ambiguous_unbalanced_agg_count    0
sentence_count                                           0
dtype: int64


In [111]:
from datetime import datetime
file_name = datetime.now().strftime('../../data/democracy_aggregate_counts_for_dimensions_%d%m%y')
print(f"Saving {file_name}.csv")
df_agg_all.to_csv(f"{file_name}.csv", index=False)
print(f"Saving {file_name}.xlsx")
df_agg_all.to_excel(f"{file_name}.xlsx", index=False)

Saving ../../data/democracy_aggregate_counts_for_dimensions_020824.csv
Saving ../../data/democracy_aggregate_counts_for_dimensions_020824.xlsx


In [110]:
test_df = pd.read_csv(f"{file_name}.csv")
print(test_df.columns)
any_nulls = test_df.isnull().values.any()

print("Any null values:", any_nulls)
# Get the count of null values in each column
null_counts = test_df.isnull().sum()

print("Count of null values in each column:\n", null_counts)

Index(['country', 'year', 'source', 'predicted_dim_roberta_unbalanced',
       'dim_roberta_unbalanced_agg_count',
       'predicted_dim_roberta_no_ambiguous_unbalanced',
       'dim_roberta_no_ambiguous_unbalanced_agg_count',
       'dim_keyword_dictionary', 'dim_keyword_dictionary_agg_count',
       'sentence_count'],
      dtype='object')
Any null values: False
Count of null values in each column:
 country                                          0
year                                             0
source                                           0
predicted_dim_roberta_unbalanced                 0
dim_roberta_unbalanced_agg_count                 0
predicted_dim_roberta_no_ambiguous_unbalanced    0
dim_roberta_no_ambiguous_unbalanced_agg_count    0
dim_keyword_dictionary                           0
dim_keyword_dictionary_agg_count                 0
sentence_count                                   0
dtype: int64
